# Dashboard Explicativo de Clustering de Clientes

Este dashboard permite analizar la segmentación de clientes por patrones de compra, diversidad de pago y preferencias de productos, facilitando la interpretación y toma de decisiones comerciales.

### Cómo usar este dashboard
- Utiliza el filtro de cluster para explorar comportamientos por segmento.
- Pasa el mouse sobre los títulos para ver explicaciones (icono ℹ️).
- Las tablas y gráficas se actualizan según el cluster seleccionado.
- Lee las descripciones junto a cada panel para entender cómo interpretar los resultados.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, NMF
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc, html, Input, Output, dash_table
import dash_bootstrap_components as dbc
from scipy.stats import entropy
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 1. Carga de datos
file_path = "Data_Set_Global.xlsx"
pedidos = pd.read_excel(file_path, sheet_name="Pedidos")
pedidos["total_price"] = pedidos["quantity"] * pedidos["unit_price"]
pedidos["order_date"] = pd.to_datetime(pedidos["order_date"])
pedidos["promised_delivery_time"] = pd.to_datetime(pedidos["promised_delivery_time"])
pedidos["actual_delivery_time"] = pd.to_datetime(pedidos["actual_delivery_time"])
pedidos["delivery_delay_min"] = (
    (pedidos["actual_delivery_time"] - pedidos["promised_delivery_time"]).dt.total_seconds() / 60
)
pedidos = pedidos.dropna(subset=["order_id", "customer_id", "order_date"]).copy()

In [ ]:
# 2. KPIs generales
def calcular_kpis(df):
    return {
        "Total Clientes": df["customer_id"].nunique(),
        "Total Órdenes": df["order_id"].nunique(),
        "Total Productos": df["product_id"].nunique(),
        "Facturación Total": df["total_price"].sum(),
        "Periodos Analizados": f"{df['order_date'].min().date()} a {df['order_date'].max().date()}"
    }
kpi_dict = calcular_kpis(pedidos)

In [ ]:
# 3. EDA visual y descriptivo
def eda_visual(df):
    df_seg = df.groupby("customer_segment")["order_id"].count().reset_index()
    fig_bar_segment = px.bar(df_seg, x="customer_segment", y="order_id", text="order_id",
                             labels={"order_id": "# Órdenes"}, title="Órdenes por Segmento de Cliente")
    # Top 5 productos por segmento
    N = 5
    topN = (
        df.groupby(["customer_segment", "product_name"])
        .size().groupby(level=0, group_keys=False).nlargest(N)
        .reset_index(name="compras")
    )
    top5_prod_segment = {}
    for seg in topN["customer_segment"].unique():
        top5_prod_segment[seg] = topN[topN["customer_segment"] == seg][["product_name", "compras"]].to_dict('records')
    # Correlación de numéricas
    num_cols = df.select_dtypes("number").columns
    corr = df[num_cols].corr()
    fig_corr = px.imshow(corr, color_continuous_scale='RdBu', title="Matriz de Correlación Variables Numéricas")
    # Histograma de demoras
    fig_delay = px.histogram(df, x="delivery_delay_min", nbins=50, title="Demora de entrega (min)")
    # Conversión: días a primera compra
    agg = (
        df.groupby("customer_id")
        .agg(
            first_order=("order_date", "min"),
            registration_date=("registration_date", "first")
        ).reset_index()
    )
    agg["days_to_first"] = (agg["first_order"] - agg["registration_date"]).dt.days
    fig_conversion = px.histogram(agg, x="days_to_first", nbins=30, title="Días hasta primera compra")
    return fig_bar_segment, top5_prod_segment, fig_corr, fig_delay, fig_conversion

fig_bar_segment, top5_prod_segment, fig_corr, fig_delay, fig_conversion = eda_visual(pedidos)

In [ ]:
# 4. Variables para clustering
snapshot_date = pedidos["order_date"].max() + dt.timedelta(days=1)
rfm = pedidos.groupby("customer_id").agg(
    recency=("order_date", lambda date: (snapshot_date - date.max()).days),
    frequency=("order_id", "nunique"),
    monetary=("total_price", "sum")
)
scaler_rfm = StandardScaler()
rfm_z = pd.DataFrame(scaler_rfm.fit_transform(rfm), index=rfm.index, columns=[c+"_z" for c in rfm.columns])

# Entropía de pago
pay_counts = pedidos.groupby(["customer_id", "payment_method"]).size().unstack(fill_value=0)
pay_probs = pay_counts.div(pay_counts.sum(axis=1), axis=0)
payment_entropy = pay_probs.apply(lambda row: entropy(row, base=2), axis=1)
customer_payment_diversity = payment_entropy.rename("payment_entropy").reset_index()

# NMF tópicos cliente-producto
cust_prod = pedidos.groupby(["customer_id", "product_id"]).size().unstack(fill_value=0)
K = 10
nmf = NMF(n_components=K, init="random", random_state=42)
W = nmf.fit_transform(cust_prod)
H = nmf.components_
customer_topics = pd.DataFrame(W, index=cust_prod.index, columns=[f"topic_{i+1}" for i in range(K)])

# Matriz final para clustering
full_df = (
    rfm_z
    .join(customer_payment_diversity.set_index("customer_id"))
    .join(customer_topics)
).fillna(0)
X_scaled = StandardScaler().fit_transform(full_df.select_dtypes(include=np.number))

In [ ]:
# 5. Clustering final
K_OPT = 3
kmeans = KMeans(n_clusters=K_OPT, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_scaled)
full_df["cluster"] = cluster_labels
full_df.index.name = "customer_id"
profile_cols = [c for c in full_df.columns if c.startswith(('recency','frequency','monetary','payment_entropy','topic_'))]
cluster_profile = (
    full_df.groupby("cluster")[profile_cols]
    .mean().round(2)
)
cluster_size = full_df["cluster"].value_counts().sort_index()

In [ ]:
# 6. Interpretación de tópicos
product_ids = cust_prod.columns
prod_meta = pd.read_excel(file_path, sheet_name="Productos").set_index("product_id")
top_words_topic = {}
for i, topic in enumerate(H):
    top_ids = np.argsort(topic)[-5:][::-1]
    top_names = prod_meta.loc[product_ids[top_ids], "product_name"].tolist()
    top_words_topic[f"topic_{i+1}"] = top_names

In [ ]:
# 7. Dashboard interactivo y didáctico
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.MINTY])

def tooltip(id, text):
    return dbc.Tooltip(text, target=id, placement="top", style={"fontSize":"0.95em", "maxWidth":"300px"})

app.layout = dbc.Container([
    html.H2("Dashboard de Segmentación de Clientes", style={"marginTop": 15}),
    html.Hr(),
    dbc.Alert([
        html.H5("¿Cómo usar este dashboard?", style={"fontWeight": "bold"}),
        html.Ul([
            html.Li("Utiliza el filtro de cluster para explorar comportamientos por segmento."),
            html.Li("Pasa el mouse sobre los títulos para ver explicaciones (icono ℹ️)."),
            html.Li("Las tablas y gráficas se actualizan según el cluster seleccionado."),
            html.Li("Lee las descripciones junto a cada panel para entender cómo interpretar los resultados."),
        ]),
        html.Br(),
        html.P("Este dashboard permite analizar la segmentación de clientes por patrones de compra, diversidad de pago y preferencias de productos, facilitando la interpretación y toma de decisiones comerciales.")
    ], color="info", dismissable=True, is_open=True, style={"marginBottom": "20px"}),

    # Filtro global de cluster
    dbc.Row([
        dbc.Col([
            html.Label([
                "Filtrar por cluster/clústeres:",
                html.Span(" ℹ️", id="tt-cluster-filter", style={"cursor":"pointer"}),
            ]),
            dcc.Dropdown(
                id='global-cluster-filter',
                options=[{"label": f"Cluster {c}", "value": c} for c in sorted(full_df["cluster"].unique())],
                value=[], multi=True, placeholder="Elige uno o más clusters para filtrar el análisis",
                style={"marginBottom": "10px"}
            ),
            tooltip("tt-cluster-filter", "Selecciona uno o más clusters para filtrar todas las gráficas y KPIs del dashboard. Útil para comparar segmentos específicos.")
        ], width=6)
    ]),
    # KPIs filtrados
    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader(k),
                dbc.CardBody(html.H5(id=f"kpi-{i}"))
            ]), width=3
        ) for i, k in enumerate(kpi_dict.keys())
    ]),

    html.Br(),
    html.H3([
        "Análisis Exploratorio de los Datos (EDA)",
        html.Span(" ℹ️", id="tt-eda", style={"cursor":"pointer", "fontSize": "1em"})
    ]),
    tooltip("tt-eda", "Explora patrones de compra, productos más vendidos, correlaciones y tiempos clave. Todos los gráficos se filtran por cluster si aplicas el filtro global."),
    html.P("Visualizaciones clave para entender el comportamiento de los clientes y el contexto previo al clustering."),
    dbc.Row([
        dbc.Col([dcc.Graph(id="fig-bar-segment")], width=6),
        dbc.Col([dcc.Graph(id="fig-corr")], width=6),
    ]),
    dbc.Row([
        dbc.Col([dcc.Graph(id="fig-delay")], width=6),
        dbc.Col([dcc.Graph(id="fig-conversion")], width=6),
    ]),
    html.Br(),
    html.H5([
        "Top 5 productos por segmento de cliente",
        html.Span(" ℹ️", id="tt-topprod", style={"cursor":"pointer", "fontSize": "1em"})
    ]),
    tooltip("tt-topprod", "Para cada segmento, se muestran los productos más comprados por sus clientes. Ayuda a identificar preferencias específicas por cluster."),
    html.Div(id="top5-prod-list"),
    html.Hr(),
    html.H3([
        "Clustering & Segmentación de Clientes",
        html.Span(" ℹ️", id="tt-clustering", style={"cursor":"pointer", "fontSize": "1em"})
    ]),
    tooltip("tt-clustering", "Se aplicó KMeans sobre variables de comportamiento. El gráfico PCA muestra la separación de clusters en el espacio reducido."),
    html.P("Experimenta filtrando clusters para ver cómo cambian los resultados. Observa el tamaño y perfil de cada segmento."),
    dcc.Graph(id="fig-pca"),
    html.H5("Tamaño de cada cluster"),
    dcc.Graph(id="fig-cluster-size"),
    html.Hr(),
    html.H4([
        "Resumen descriptivo de clusters",
        html.Span(" ℹ️", id="tt-profile", style={"cursor":"pointer", "fontSize": "1em"})
    ]),
    tooltip("tt-profile", "Valores promedio de variables relevantes por cluster. Útil para comparar características clave entre segmentos."),
    html.P("La tabla muestra los valores promedio de cada variable (RFM, entropía y tópicos) en cada cluster."),
    dash_table.DataTable(
        id="cluster-profile-table",
        columns=[{"name": i, "id": i} for i in cluster_profile.columns],
        style_table={'width': '100%'}, style_cell={'textAlign': 'center'},
        page_size=10
    ),
    html.Br(),
    html.H4([
        "Distribución de tópicos en clusters (boxplot interactivo)",
        html.Span(" ℹ️", id="tt-topicbox", style={"cursor":"pointer", "fontSize": "1em"})
    ]),
    tooltip("tt-topicbox", "Selecciona un tópico para explorar cómo varía entre clusters. Los tópicos resumen patrones de consumo comunes extraídos automáticamente."),
    dcc.Dropdown(
        id='topic-dropdown',
        options=[{"label": t, "value": t} for t in customer_topics.columns],
        value=customer_topics.columns[0], clearable=False
    ),
    dcc.Graph(id='topic-box'),
    html.Hr(),
    html.H4([
        "Interpretación de tópicos NMF",
        html.Span(" ℹ️", id="tt-nmf", style={"cursor":"pointer", "fontSize": "1em"})
    ]),
    tooltip("tt-nmf", "Cada tópico representa un patrón de consumo distinto. La tabla muestra los productos más representativos de cada tópico."),
    html.P("Los tópicos resumen agrupaciones automáticas de productos que suelen ser comprados juntos."),
    dash_table.DataTable(
        columns=[{"name": k, "id": k} for k in top_words_topic.keys()],
        data=[{k: ', '.join(v) for k, v in top_words_topic.items()}],
        style_table={'width': '100%'}, style_cell={'textAlign': 'left'},
    ),
    html.Hr(),
    html.H4([
        "Detalle de clientes (filtrable por cluster)",
        html.Span(" ℹ️", id="tt-clients", style={"cursor":"pointer", "fontSize": "1em"})
    ]),
    tooltip("tt-clients", "Navega por los clientes de cada cluster. Puedes filtrar y ordenar por cualquier variable."),
    dash_table.DataTable(
        id='clientes-table',
        columns=[{"name": i, "id": i} for i in ["recency_z","frequency_z","monetary_z","payment_entropy"]+list(customer_topics.columns)+["cluster"]],
        page_size=12, filter_action="native", sort_action="native", style_table={'overflowX': 'auto'}, style_cell={'textAlign': 'center'}
    ),
], fluid=True)

# CALLBACKS
@app.callback(
    [Output(f"kpi-{i}", "children") for i in range(len(kpi_dict))],
    Output("fig-bar-segment", "figure"),
    Output("top5-prod-list", "children"),
    Output("fig-corr", "figure"),
    Output("fig-delay", "figure"),
    Output("fig-conversion", "figure"),
    Output("fig-pca", "figure"),
    Output("fig-cluster-size", "figure"),
    Output("cluster-profile-table", "data"),
    Output("topic-box", "figure"),
    Output("clientes-table", "data"),
    Input("global-cluster-filter", "value"),
    Input("topic-dropdown", "value")
)
def update_all(cluster_vals, topic):
    # Filtrado por cluster
    if cluster_vals:
        mask = full_df["cluster"].isin(cluster_vals)
        pedidos_filt = pedidos[pedidos["customer_id"].isin(full_df[mask].index)]
        full_df_filt = full_df[mask]
    else:
        pedidos_filt = pedidos.copy()
        full_df_filt = full_df.copy()
    # KPIs
    kpi_vals = list(calcular_kpis(pedidos_filt).values())
    # EDA
    fig_bar_segment, top5_prod_segment, fig_corr, fig_delay, fig_conversion = eda_visual(pedidos_filt)
    # Lista top5
    top5_list = html.Ul([
        html.Li([
            html.B(f"Segmento {seg}: "), 
            ', '.join([f"{d['product_name']} ({d['compras']})" for d in lst])
        ]) for seg, lst in top5_prod_segment.items()
    ])
    # PCA plot
    if not full_df_filt.empty:
        pca_2d = PCA(n_components=2).fit_transform(StandardScaler().fit_transform(full_df_filt[profile_cols]))
        fig_pca = px.scatter(
            x=pca_2d[:,0], y=pca_2d[:,1], color=full_df_filt["cluster"].astype(str),
            title="Clusters en espacio PCA (2D)", labels={"x": "PCA1", "y": "PCA2", "color": "Cluster"},
            hover_data={"customer_id": full_df_filt.index}
        )
    else:
        fig_pca = go.Figure()
    # Cluster size
    csize = full_df_filt["cluster"].value_counts().sort_index()
    fig_csize = px.bar(x=csize.index.astype(str), y=csize.values, labels={'x':'Cluster','y':'#Clientes'}, title="Tamaño de cada cluster")
    # Profile table
    profile_data = full_df_filt.groupby("cluster")[profile_cols].mean().round(2).reset_index().to_dict('records')
    # Topic boxplot
    if not full_df_filt.empty:
        fig_topic = px.box(full_df_filt.reset_index(), x=full_df_filt["cluster"].astype(str), y=topic, points='all',
                        title=f"Boxplot de {topic} por cluster", labels={"x": "Cluster", "y": topic})
    else:
        fig_topic = go.Figure()
    # Clientes table
    clientes_data = full_df_filt.reset_index()[["recency_z","frequency_z","monetary_z","payment_entropy"]+list(customer_topics.columns)+["cluster"]].to_dict('records')
    return kpi_vals, fig_bar_segment, top5_list, fig_corr, fig_delay, fig_conversion, fig_pca, fig_csize, profile_data, fig_topic, clientes_data

# Para lanzar el dashboard, descomenta y ejecuta:
# app.run(debug=True, port=8051)
# Y abre: http://127.0.0.1:8051/